In [314]:
library(tidyverse)
library(nycflights13)
library(lubridate)

# Lecture 11: relational data II

## Filtering joins

Filtering joins allow use to filter rows on one table based on their presence or absence in another table. We've already seen some examples of this on the problem sets:

In [15]:
top6 = count(flights, carrier) %>% top_n(6)
flights_top6 = filter(flights, carrier %in% top6$carrier) %>% print

Selecting by n


# A tibble: 274,709 x 19
    year month   day dep_t… sched… dep_d… arr_… sched… arr_d… carr… flig… tail…
   <int> <int> <int>  <int>  <int>  <dbl> <int>  <int>  <dbl> <chr> <int> <chr>
 1  2013     1     1    517    515   2.00   830    819  11.0  UA     1545 N142…
 2  2013     1     1    533    529   4.00   850    830  20.0  UA     1714 N242…
 3  2013     1     1    542    540   2.00   923    850  33.0  AA     1141 N619…
 4  2013     1     1    544    545  -1.00  1004   1022 -18.0  B6      725 N804…
 5  2013     1     1    554    600  -6.00   812    837 -25.0  DL      461 N668…
 6  2013     1     1    554    558  -4.00   740    728  12.0  UA     1696 N394…
 7  2013     1     1    555    600  -5.00   913    854  19.0  B6      507 N516…
 8  2013     1     1    557    600  -3.00   709    723 -14.0  EV     5708 N829…
 9  2013     1     1    557    600  -3.00   838    846 - 8.00 B6       79 N593…
10  2013     1     1    558    600  -2.00   753    745   8.00 AA      301 N3AL…
# ... with 274,

What if we wanted to filter based on more than one variable? For example, based on busiest days of the year:

In [27]:
top6_days = count(flights, year, month, day) %>% top_n(6) %>% print

Selecting by n


# A tibble: 7 x 4
   year month   day     n
  <int> <int> <int> <int>
1  2013     7     8  1004
2  2013     7    10  1004
3  2013     7    11  1006
4  2013     7    18  1003
5  2013     7    25  1003
6  2013    11    27  1014
7  2013    12     2  1004


One option would be to construct a new string variable which we could then match using the same technique:

In [47]:
top6_days_ymd = unite(top6_days, ymd, year:day, sep="-") %>% .$ymd
flights_ymd = unite(flights, ymd, year:day, sep="-")
filter(flights_ymd, ymd %in% top6_days_ymd) %>% print

# A tibble: 7,038 x 17
   ymd    dep_t… sche… dep_de… arr_… sche… arr_d… carr… flig… tail… orig… dest 
   <chr>   <int> <int>   <dbl> <int> <int>  <dbl> <chr> <int> <chr> <chr> <chr>
 1 2013-…     28  1930  298      129  2049  280   EV     5769 N833… LGA   IAD  
 2 2013-…    505   500    5.00   703   651   12.0 US     1895 N198… EWR   CLT  
 3 2013-…    514   515 -  1.00   743   808 - 25.0 UA     1096 N668… EWR   IAH  
 4 2013-…    535   540 -  5.00   907   850   17.0 AA     2243 N5CK… JFK   MIA  
 5 2013-…    540   545 -  5.00   815   835 - 20.0 UA     1014 N372… LGA   IAH  
 6 2013-…    551   600 -  9.00   920   854   26.0 B6      605 N589… EWR   FLL  
 7 2013-…    551   600 -  9.00   918   849   29.0 B6      353 N655… JFK   PBI  
 8 2013-…    554   600 -  6.00   719   719    0   EV     5747 N820… LGA   IAD  
 9 2013-…    554   600 -  6.00   707   717 - 10.0 EV     5716 N832… JFK   IAD  
10 2013-…    554   600 -  6.00   907   846   21.0 B6      583 N507… JFK   MCO  
# ... with 7,028 

But this is cumbersome and will also be slow if you are matching to a large list of choices.

 A better option is to use the built-in command `semi_join`. `semi_join(x, y)` keeps all the observations in `x` that are also in `y`.
![semi join](http://r4ds.had.co.nz/diagrams/join-semi.png)

`semi_join()` takes the same `key=` options as the other join commands.

In [31]:
semi_join(flights, top6_days) %>% print

Joining, by = c("year", "month", "day")


# A tibble: 7,038 x 19
    year month   day dep_t… sched… dep_de… arr_… sche… arr_d… carr… flig… tail…
   <int> <int> <int>  <int>  <int>   <dbl> <int> <int>  <dbl> <chr> <int> <chr>
 1  2013    11    27     28   1930  298      129  2049  280   EV     5769 N833…
 2  2013    11    27    505    500    5.00   703   651   12.0 US     1895 N198…
 3  2013    11    27    514    515 -  1.00   743   808 - 25.0 UA     1096 N668…
 4  2013    11    27    535    540 -  5.00   907   850   17.0 AA     2243 N5CK…
 5  2013    11    27    540    545 -  5.00   815   835 - 20.0 UA     1014 N372…
 6  2013    11    27    551    600 -  9.00   920   854   26.0 B6      605 N589…
 7  2013    11    27    551    600 -  9.00   918   849   29.0 B6      353 N655…
 8  2013    11    27    554    600 -  6.00   719   719    0   EV     5747 N820…
 9  2013    11    27    554    600 -  6.00   707   717 - 10.0 EV     5716 N832…
10  2013    11    27    554    600 -  6.00   907   846   21.0 B6      583 N507…
# ... with 7,028 

The opposite of a semi-join is an anti-join:
![anti-join](http://r4ds.had.co.nz/diagrams/join-anti.png)

Anti-joins are useful for finding mismatches in joins. Yesterday we saw that some entries in `flights` have `tailnum`s that do not exist in the `planes` table. To find out more about these flights we could use an anti-join to select only those rows:

In [63]:
missing_planes = anti_join(flights, planes, by="tailnum") %>% filter(!is.na(tailnum)) %>% print

# A tibble: 50,094 x 19
    year month   day dep_t… sched… dep_de… arr_… sche… arr_d… carr… flig… tail…
   <int> <int> <int>  <int>  <int>   <dbl> <int> <int>  <dbl> <chr> <int> <chr>
 1  2013     1     1    558    600  - 2.00   753   745   8.00 AA      301 N3AL…
 2  2013     1     1    559    600  - 1.00   941   910  31.0  AA      707 N3DU…
 3  2013     1     1    600    600    0      837   825  12.0  MQ     4650 N542…
 4  2013     1     1    602    605  - 3.00   821   805  16.0  MQ     4401 N730…
 5  2013     1     1    608    600    8.00   807   735  32.0  MQ     3768 N9EA…
 6  2013     1     1    611    600   11.0    945   931  14.0  UA      303 N532…
 7  2013     1     1    623    610   13.0    920   915   5.00 AA     1837 N3EM…
 8  2013     1     1    624    630  - 6.00   840   830  10.0  MQ     4599 N518…
 9  2013     1     1    628    630  - 2.00  1137  1140 - 3.00 AA      413 N3BA…
10  2013     1     1    629    630  - 1.00   824   810  14.0  AA      303 N3CY…
# ... with 50,08

Do we see any pattern in this table that could explain the missing tail numbers?

In [64]:
print(missing_planes %>% sample_n(100), width=Inf)

# A tibble: 100 x 19
    year month   day dep_time sched_dep_time dep_delay arr_time sched_arr_time
   <int> <int> <int>    <int>          <int>     <dbl>    <int>          <int>
 1  2013     6     5      552            600   -  8.00      801            810
 2  2013    10     4      620            625   -  5.00      830            850
 3  2013     3    31      811            810      1.00      939            955
 4  2013     3    20      825            835   - 10.0      1056           1105
 5  2013     2    17     1443           1450   -  7.00     1618           1640
 6  2013     9    27     1450           1455   -  5.00     1804           1750
 7  2013    11     3     2022           2025   -  3.00     2136           2140
 8  2013     7    25      250           2245    245         530            135
 9  2013     1    25     1533           1535   -  2.00     1832           1850
10  2013     2    26     1054           1100   -  6.00     1203           1215
   arr_delay carrier flight tai

Interestingly, the top 5 most frequent `tailnum`s in the overall data are almost the same:

In [65]:
count(flights, tailnum) %>% top_n(5)

Selecting by n


  tailnum n   
1 N711MQ   486
2 N722MQ   513
3 N723MQ   507
4 N725MQ   575
5 NA      2512

Let's examine whatever planes in the top 10 *are* present in `planes`:

In [66]:
semi_join(planes, count(flights, tailnum) %>% top_n(10)) %>% print

Selecting by n
Joining, by = "tailnum"


# A tibble: 5 x 9
  tailnum  year type                    manufa… model   engi… seats speed engi…
  <chr>   <int> <chr>                   <chr>   <chr>   <int> <int> <int> <chr>
1 N258JB   2006 Fixed wing multi engine EMBRAER ERJ 19…     2    20    NA Turb…
2 N298JB   2009 Fixed wing multi engine EMBRAER ERJ 19…     2    20    NA Turb…
3 N351JB   2012 Fixed wing multi engine EMBRAER ERJ 19…     2    20    NA Turb…
4 N353JB   2012 Fixed wing multi engine EMBRAER ERJ 19…     2    20    NA Turb…
5 N711MQ   1976 Fixed wing multi engine GULFST… G1159B      2    22    NA Turb…


They are all small regional jets. Does this give us any clue as to why `tailnum` is missing?

In [79]:
print(missing_planes %>% sample_n(10), width=Inf)

# A tibble: 10 x 19
    year month   day dep_time sched_dep_time dep_delay arr_time sched_arr_time
   <int> <int> <int>    <int>          <int>     <dbl>    <int>          <int>
 1  2013     4    22      855            855      0        1204           1120
 2  2013    11    29      851            830     21.0      1102           1100
 3  2013     7    12     1450           1429     21.0      1907           1659
 4  2013     5    28      658            700    - 2.00      935            950
 5  2013     7    31      659            710    -11.0       910            915
 6  2013     5    26     2101           2105    - 4.00     2246           2305
 7  2013     3    26     1454           1500    - 6.00     1827           1825
 8  2013     4     1      555            600    - 5.00      834            825
 9  2013     2    19      609            610    - 1.00      902            910
10  2013     3    16      727            730    - 3.00     1018           1040
   arr_delay carrier flight tail

In [98]:
count(missing_planes, carrier)
flights_planes = filter(flights, !is.na(tailnum)) %>% 
    left_join(planes, by="tailnum", suffix=c("_fl", "_pl"))
tab = table(flights_planes$carrier %in% c("MQ", "AA"), is.na(flights_planes$year_pl)) %>% print
prop.table(tab, 2)

  carrier n    
1 AA      22474
2 B6        830
3 DL        110
4 F9         47
5 FL        187
6 MQ      25395
7 UA       1007
8 US         36
9 WN          8

       
         FALSE   TRUE
  FALSE 267885   7339
  TRUE   10979  48061


       
             FALSE       TRUE
  FALSE 0.96062955 0.13247292
  TRUE  0.03937045 0.86752708

## Set functions
Finally we will study set functions. These take two tables `x` and `y` and treat their rows as elements of a set. Then, we can take the intersection, union, etc. In order for this to make sense, the `x` and `y` inputs must have the same variables.

In [296]:
df1 <- tribble(
  ~x, ~y,
   1,  1,
   2,  1
)
df2 <- tribble(
  ~x, ~y,
   1,  1,
   1,  2
)
df1
df2

  x y
1 1 1
2 2 1

  x y
1 1 1
2 1 2

The intersection of `df1` and `df2` will give me all the rows that are found in both tables:

In [302]:
dplyr::intersect(df1, df2)

  x y
1 1 1

The union will contain (one copy of) each row that is found in either table.

In [308]:
dplyr::union(df1, df2)  # note 3 rows not 4

  x y
1 1 1
2 2 1
3 1 1
4 1 2

If you really want all rows (including multiplicity), use `union_all()` or `row_bind()`:

In [312]:
dplyr::union_all(df1, df2)
bind_rows(df1, df2)

  x y
1 1 1
2 2 1
3 1 1
4 1 2

  x y
1 1 1
2 2 1
3 1 1
4 1 2

Finally, the set difference is all rows that are found in exactly one of the two tables. (Like xor for sets.)

In [313]:
dplyr::setdiff(df1, df2)

  x y
1 2 1